In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feedback or issues?
For any feedback or questions, please open an [issue](https://github.com/googleapis/python-aiplatform/issues).

# Vertex SDK for Python: AutoML Image Classfication Training with Customer Managed Encryption Keys (CMEK) Example
To use this Jupyter notebook, create a copy of the notebook in Colab and open it. You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Colab automatically displays the return value of the last line in each cell.

This notebook demonstrate how to train an AutoML Image Classification model with CMEK. It will require you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

# Install SDK
 
After the SDK installation the kernel will be automatically restarted.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install --upgrade google-cloud-kms
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
REGION = "YOUR REGION"  # e.g. us-central1
MY_PROJECT = "YOUR PROJECT ID"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as ucaip

## Setting up Customer Managed Encryption Keys

By default, Google Cloud automatically encrypts data when it is at rest using encryption keys managed by Google. If you have specific compliance or regulatory requirements related to the keys that protect your data, you can use customer-managed encryption keys (CMEK) for your training jobs.

For more info on using CMEK on Vertex AI, please see: [https://cloud.google.com/vertex-ai/docs/general/cmek#before_you_begin](https://cloud.google.com/vertex-ai/docs/general/cmek#before_you_begin)

You can create a key using the guide above or executing the the Notebook cells below.

1. Register your application for Cloud Key Management Service (KMS) API in Google Cloud Platform at https://console.cloud.google.com/flows/enableapi?apiid=cloudkms.googleapis.com

2. Create a key ring

Create a key ring

In [ ]:
KEY_RING_ID = "your_key_ring_name"


# Reference: https://cloud.google.com/kms/docs/samples/kms-create-key-ring
def create_key_ring(project_id, location_id, id):
    """
    Creates a new key ring in Cloud KMS

    Args:
        project_id (string): Google Cloud project ID (e.g. 'my-project').
        location_id (string): Cloud KMS location (e.g. 'us-east1').
        id (string): ID of the key ring to create (e.g. 'my-key-ring').

    Returns:
        KeyRing: Cloud KMS key ring.

    """

    # Import the client library.
    from google.cloud import kms

    # Create the client.
    client = kms.KeyManagementServiceClient()

    # Build the parent location name.
    location_name = f"projects/{project_id}/locations/{location_id}"

    # Build the key ring.
    key_ring = {}

    # Call the API.
    created_key_ring = client.create_key_ring(
        request={"parent": location_name, "key_ring_id": id, "key_ring": key_ring}
    )
    print("Created key ring: {}".format(created_key_ring.name))
    return created_key_ring


create_key_ring(project_id=MY_PROJECT, location_id=REGION, id=KEY_RING_ID)

Create a key

In [ ]:
KEY_ID = "your_key_id"


# Reference: https://cloud.google.com/kms/docs/samples/kms-create-key-symmetric-encrypt-decrypt
def create_key_symmetric_encrypt_decrypt(project_id, location_id, key_ring_id, id):
    """
    Creates a new symmetric encryption/decryption key in Cloud KMS.

    Args:
        project_id (string): Google Cloud project ID (e.g. 'my-project').
        location_id (string): Cloud KMS location (e.g. 'us-east1').
        key_ring_id (string): ID of the Cloud KMS key ring (e.g. 'my-key-ring').
        id (string): ID of the key to create (e.g. 'my-symmetric-key').

    Returns:
        CryptoKey: Cloud KMS key.

    """

    # Import the client library.
    from google.cloud import kms

    # Create the client.
    client = kms.KeyManagementServiceClient()

    # Build the parent key ring name.
    key_ring_name = client.key_ring_path(project_id, location_id, key_ring_id)

    # Build the key.
    purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
    algorithm = (
        kms.CryptoKeyVersion.CryptoKeyVersionAlgorithm.GOOGLE_SYMMETRIC_ENCRYPTION
    )
    key = {
        "purpose": purpose,
        "version_template": {
            "algorithm": algorithm,
        },
    }

    # Call the API.
    created_key = client.create_crypto_key(
        request={"parent": key_ring_name, "crypto_key_id": id, "crypto_key": key}
    )
    print("Created symmetric key: {}".format(created_key.name))
    return created_key


create_key_symmetric_encrypt_decrypt(
    project_id=MY_PROJECT, location_id=REGION, key_ring_id=KEY_RING_ID, id=KEY_ID
)

Give permissions to key to the Vertex AI service account

In [ ]:
# Reference: https://cloud.google.com/vertex-ai/docs/general/cmek#granting_permissions
# Get the service account
SERVICE_ACCOUNT = ! gcloud projects get-iam-policy {MY_PROJECT} \
  --flatten="bindings[].members" \
  --format="table(bindings.members)" \
  --filter="bindings.role:roles/aiplatform.serviceAgent" \
  | grep -oP "service-.+?@gcp-sa-aiplatform.iam.gserviceaccount.com"
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]

print(f"Service account is: {SERVICE_ACCOUNT}")

# Give permissions
!gcloud kms keys add-iam-policy-binding {KEY_ID} \
  --keyring={KEY_RING_ID} \
  --location={REGION} \
  --project={MY_PROJECT} \
  --member=serviceAccount:{SERVICE_ACCOUNT} \
  --role=roles/cloudkms.cryptoKeyEncrypterDecrypter

In [ ]:
# Create the full resource identifier for the created key
ENCRYPTION_SPEC_KEY_NAME = f"projects/{MY_PROJECT}/locations/{REGION}/keyRings/{KEY_RING_ID}/cryptoKeys/{KEY_ID}"

## Initialize Vertex SDK for Python

Initialize the *client* for Vertex AI

All resources created during this Notebook run will encrypted with the encryption key created above.

You can override the encryption key at each function call.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(
    project=MY_PROJECT,
    staging_bucket=MY_STAGING_BUCKET,
    location=REGION,
    encryption_spec_key_name=ENCRYPTION_SPEC_KEY_NAME,
)

# Create Managed Image Dataset from CSV

This section will create a managed Image dataset from the Flowers dataset. For more imformation on this dataset please visit https://www.tensorflow.org/datasets/catalog/tf_flowers.

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

ds = aiplatform.ImageDataset.create(
    display_name="flowers",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

ds.resource_name

# Launch a Training Job to Create a Model

Train an AutoML Image Classification model.

In [ ]:
job = aiplatform.AutoMLImageTrainingJob(
    display_name="train-iris-automl-mbsdk-1",
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

# This will take around half an hour to run
model = job.run(
    dataset=ds,
    model_display_name="iris-classification-model-mbsdk",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

# Deploy Your Model

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [ ]:
endpoint = model.deploy()

# Predict on Endpoint
- Take one sample from the data imported to the dataset
- This sample will be encoded to base64 and passed to the endpoint for prediction

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances_list)

prediction

# Undeploy Model from Endpoint

In [ ]:
endpoint.undeploy_all()